# Procesamiento en Python acerca del Dataframe 1: Actividad de clientes (actividad_clientes.csv)

In [36]:
# Importar librerías necesarias

import pandas as pd
import numpy as np
import plotly as pl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msmo
from statistics import multimode, mean, median
import math

# Importar datos del archivo .csv

In [37]:
df = pd.read_csv("..\\dataset\\actividad_clientes.csv", index_col=0)
df.head()

,rental_date,return_date,first_name,last_name,district,postal_code,city,country,amount
rental_id,,,,,,,,,
76,2005-05-25 11:30:37,2005-06-03 12:00:37,mary,smith,nagasaki,35200,sasebo,japan,2.99
573,2005-05-28 10:35:23,2005-06-03 06:32:23,mary,smith,nagasaki,35200,sasebo,japan,0.99
1185,2005-06-15 00:54:12,2005-06-23 02:42:12,mary,smith,nagasaki,35200,sasebo,japan,5.99
1422,2005-06-15 18:02:53,2005-06-19 15:54:53,mary,smith,nagasaki,35200,sasebo,japan,0.99
1476,2005-06-15 21:08:46,2005-06-25 02:26:46,mary,smith,nagasaki,35200,sasebo,japan,9.99


# Vemos las propiedades de nuestra tabla

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15861 entries, 76 to 15725
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rental_date  15861 non-null  object 
 1   return_date  15861 non-null  object 
 2   first_name   15861 non-null  object 
 3   last_name    15861 non-null  object 
 4   district     15761 non-null  object 
 5   postal_code  15861 non-null  int64  
 6   city         15861 non-null  object 
 7   country      15861 non-null  object 
 8   amount       15861 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.2+ MB


# Vemos cauntos valores nulos tenemos

In [39]:
df.isna().sum()

rental_date      0
return_date      0
first_name       0
last_name        0
district       100
postal_code      0
city             0
country          0
amount           0
dtype: int64

# Creamos una copia y eliminamos los datos nulos

In [40]:
df_copy = df.copy()

In [41]:
df_copy = df_copy.dropna()

In [42]:
df_copy.isna().sum()

rental_date    0
return_date    0
first_name     0
last_name      0
district       0
postal_code    0
city           0
country        0
amount         0
dtype: int64

# Cambiar los tipos de datos de fecha

In [43]:
# Supongamos que tu DataFrame se llama df
df_copy['rental_date'] = pd.to_datetime(df_copy['rental_date']).dt.date
df_copy['return_date'] = pd.to_datetime(df_copy['return_date']).dt.date

# Ver el resultado
df_copy.head()

,rental_date,return_date,first_name,last_name,district,postal_code,city,country,amount
rental_id,,,,,,,,,
76,2005-05-25,2005-06-03,mary,smith,nagasaki,35200,sasebo,japan,2.99
573,2005-05-28,2005-06-03,mary,smith,nagasaki,35200,sasebo,japan,0.99
1185,2005-06-15,2005-06-23,mary,smith,nagasaki,35200,sasebo,japan,5.99
1422,2005-06-15,2005-06-19,mary,smith,nagasaki,35200,sasebo,japan,0.99
1476,2005-06-15,2005-06-25,mary,smith,nagasaki,35200,sasebo,japan,9.99


# Creación de columnas derivadas

## Creación columna "rental_days"

##### Esta columna representa el numero de dias que se ha alquilado la pelicula.

In [ ]:
df_copy['rental_days'] = (pd.to_datetime(df_copy['return_date']) - pd.to_datetime(df_copy['rental_date'])).dt.days


## Creación columna "total_amount_cumsum"

##### Esta columna representa el acumulado que gasta cada cliente. Con cada alquiler de pelicula se va sumando a la cantidad total el importe de la pelicula alquilada

In [45]:
df_copy['total_amount_cumsum'] = df_copy.groupby(['first_name','last_name'])['amount'].cumsum()


## Creación columna "long_rental"

##### Columna creada para visualizar si un alquiler es de largo periodo (si es mayor que 6).

In [46]:
df_copy['long_rental'] = df_copy['rental_days'] > 6


In [47]:
df_copy.head()

,rental_date,return_date,first_name,last_name,district,postal_code,city,country,amount,rental_days,total_amount_cumsum,long_rental
rental_id,,,,,,,,,,,,
76,2005-05-25,2005-06-03,mary,smith,nagasaki,35200,sasebo,japan,2.99,9,2.99,True
573,2005-05-28,2005-06-03,mary,smith,nagasaki,35200,sasebo,japan,0.99,6,3.98,False
1185,2005-06-15,2005-06-23,mary,smith,nagasaki,35200,sasebo,japan,5.99,8,9.97,True
1422,2005-06-15,2005-06-19,mary,smith,nagasaki,35200,sasebo,japan,0.99,4,10.96,False
1476,2005-06-15,2005-06-25,mary,smith,nagasaki,35200,sasebo,japan,9.99,10,20.95,True
